#### BIBLIOTECAS E DEPENDENCIAS

In [ ]:
import os, random, shutil, tweepy, re, nltk, glob
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize as wt
from autocorrect import spell
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate, cross_val_predict

#### BASE DE DADOS

Limpando a pasta para não ter o risco de somar ao inves de sobescrever

In [ ]:
files = glob.glob('C:/Users/higor.silva/TCC/base/true/*')
for f in files:
    os.remove(f)

Pegando aleatoriamente os arquivos da base "True" para o nosso algoritmo

In [ ]:
source= "C:/Users/higor.silva/TCC/base/txtTribuna"
dest= "C:/Users/higor.silva/TCC/base/true"
no_of_files=3000

#print("%"*25+"{ Details Of Transfer }"+"%"*25)
#print("\n\nList of Files Moved to %s :-"%(dest))

for i in range(no_of_files):    
    random_file=random.choice(os.listdir(source))
    #print("%d} %s"%(i+1,random_file))
    source_file="%s\%s"%(source,random_file)
    dest_file=dest    
    shutil.copy(source_file,dest_file)

#print("\n\n"+"$"*33+"[ Files Moved Successfully ]"+"$"*33)

In [ ]:
pasta = 'base'
labels = {'true': 1, 'fake': 0}
df = pd.DataFrame()

Na célula abaixo é feito um processo de unificação dos arquivos .txt para um unico arquivo .csv

In [ ]:
for f in ('true', 'fake'):
    path = os.path.join(pasta,f)
    for file in os.listdir (path) :
        with open(os.path.join(path, file),'r', encoding='ISO-8859-1') as infile:
            txt = infile.read()
        df = df.append([[txt, labels[f]]],ignore_index=True)
df.columns = ['noticia', 'rotulo']

In [ ]:
df.to_csv('noticias_data.csv', index=False, encoding='utf-8' )

In [ ]:
#df

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
df.groupby('rotulo').rotulo.count().plot.bar(ylim=0)
print(df)

#### TWITTER

Nesta parte lidaremos com o Twitter diretamente, onde faremos a coleta do tweet. 

In [ ]:
auth = tweepy.OAuthHandler('V3MjFGhabsXp4fyEEEaicqMPR', 'onmrKkii8JjA5840pZwdH8Rmkad6QJFBxuhr4hLnJ7jB8h01p0')
auth.set_access_token('978402798416093185-DF3ig67yuKhkQcVx7PoOBTi4w56Jh2j',
        'nLRZgOiHZGotiZ7FjzR2RkL39bMZFiMm2G7X9NeJDruQz')
api = tweepy.API(auth)

In [ ]:
def obter_tweets(usuario, limite=10):
    resultados = api.user_timeline(screen_name=usuario, count=limite, tweet_mode='extended')
    tweets = [] 
    for r in resultados:
        tweet = re.sub(r'http\S+', '', r.full_text)
        tweets.append(tweet.replace('\n', ' ')) 
    return tweets 
tweets2 = obter_tweets(usuario='@GuilhermeBoulos', limite=15)

In [ ]:
#print(tweets2)

Unificando os tweets recolhidos com a nossa base de apredizado, pois todos passarão juntos pale parte de tratamento de texto e Bag Of Word

In [ ]:
df2 = pd.DataFrame(tweets2)
df2.columns = ['noticia']
#print(df2)

In [ ]:
frames = pd.concat([df, df2],ignore_index=True, sort = False)
frames = frames.replace(np.nan, 0)
frames

In [ ]:
frames.iloc[6254:6269,0]

#### PRÉ-PROSSEMANTO DE TEXTO

Nesta segunda parte é onde é feita o pré processamento do texto. Aqui é aplicado o Stop Words e Stemming/lematização. 

In [ ]:
stemmer = nltk.stem.RSLPStemmer()
arq_stopwords = open("ListaStopWords.txt", "r", encoding='utf-8')
stopwords = arq_stopwords.read()

In [ ]:
#print(stopwords)

In [ ]:
def aplicastemmer(texto):    
    frases_processed = []    
    for palavra in texto:
        comstemming = [str(stemmer.stem(p)) for p in palavra.split() if p not in stopwords]
        frases_processed.append(comstemming)            
    return frases_processed

In [ ]:
noticiario = frames['noticia'].str.lower()
noticia = aplicastemmer(noticiario)

In [ ]:
#print(noticia)

#### BAG OF WORDS

Nesta parte é feita a bag of words, onde o algoritmo a baixo pega nosso documento(após o stemming) e converte para uma lista de vetores de incidencia, na qual pontua as palavras que mais se repete. 

In [ ]:
matrix = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False)

X = matrix.fit_transform(noticia).toarray()
Y = frames['rotulo']

In [ ]:
X.shape, Y.shape

In [ ]:
X

#### TREINAMENTO

Nesta parte é feito o treinamento do algoritmo com a nossa base. O algoritmo escolhido para efetuar o treinamento é o KNN em cima da metrica do cosseno. 

E definido a quantidade de K-vizinhos e logo a seguir é feita a separação em porcentagem da base teste e treino. 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.25)

In [ ]:
knn = KNeighborsClassifier(metric = 'cosine', n_neighbors=17)

In [ ]:
knn.fit(x_train,y_train)
pred = knn.predict(x_test)

In [ ]:
#pred2 = knn.predict(X[6242:6256])

In [ ]:
#print(pred2)

In [ ]:
twitter = frames.iloc[6254:6269,0]
text_features = X[6254:6269]
predictions = knn.predict(text_features)

for text, predicted in zip(twitter, predictions):            
    print('"{}"'.format(text))
    print("  - Predicted as: '{}'".format(predicted))
    print("")

In [ ]:
result_pont = metrics.classification_report(y_test,pred,target_names = [ 'false','true'])

In [ ]:
print(result_pont)

#### CROSS - VALIDATE

In [ ]:
VAL = cross_validate(knn, X, Y, cv=10, return_train_score=True, scoring=['f1','precision','recall'])

In [ ]:
print(VAL)